# # Chapter 2 Multi-armed Bandits

### Important Points
* True value of the action is the mean reward when the action is taken overtime
* True value can be found using the expectation formula
* For k-armed bandit problem, each K actions has an expected/mean reward := Value of the action ($q_*(a)$)
* $q_*(a) =  E [R_t|A_t=a]$, is the value but in most of the cases we don't know this value
* So we use Action-Value methods <- Esitmates the value of the action and use it to take decision.
* $Q_t(a)=\frac{\sum_{i=1}^{t-1} R_i \cdot 1_{A_i=a}}{\sum_{i=1}^{t-1}1_{A_i=a}}$
* Alternatively, $Q_{n+1} = \frac{\sum_{i=1}^{n} R_i}{n}$
* Now based on the $Q_t(a)$ at time step t, we can choose the best action as $A_t =  argmax_a Q_t(a)$
* We can expand the alternative defination of expected value as $Q_{n+1} = Q_n + \frac{R_n-Q_n}{n}$
* In general form, newEstimate <- oldEstimate + stepSize [Target - oldEstimate]


### Simple Bandit Algorithm Pseudocode
* For all actions, initialize $Q(a)\leftarrow 0, N(a)\leftarrow 0$ where N(a) is the number of times the action a has been selected
* While True:
    + $A \leftarrow argmax_a Q(a)$ with probability 1-$\epsilon$ or a random action with probability $\epsilon$
    + $R \leftarrow bandit(A)$ // Reward signal from the environment/bandit
    + $N(A)\leftarrow N(A) + 1$
    + $Q(A)\leftarrow Q(A) + \frac{R - Q(A)}{N(A)}$